In [ ]:
import os
from IPython.display import Image as img

img(filename = '../input/imaages/1.png')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from scipy.stats import kstest
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler

Для виконання завдань курсової роботи був обраний датасет, що містить дані про витрати на різні етапи розвитку стартапа, та його чистий прибуток. На основі цих даних планується прогнозувати прибуток, в залежності від деякого набору параметрів, тому задача зводиться до задачі **регресії**.

Датасет для подальшого опрацювання виглядає наступним чином:

In [ ]:
data = pd.read_csv('../input/startup-logistic-regression/50_Startups.csv')
data.head(6)

Одразу можна помітити, що категорії State потребують переведення у числовий формат, оскільки використовувати текстові дані для навчання моделі неможливо. Кодування текстових категорій відбувається за допомогою **LabelEncoder()**.

[https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html](http://)

Датасет після викорастання **LabelEncoder()**:

In [ ]:
data['State'] = LabelEncoder().fit_transform(data['State'])
data.head(6)

# 1.	Проаналізувати набір даних на наявність пропущених значень та NaN. У випадку наявності таких значень замінити їх за допомогою методу ковзного вікна.

Для виявлення пропущених значень та NaN використовується метод **isna()**, який повертає датафрейм тієї ж розмірності, але кожне значення замінюється на True, якщо на цьому місці стоїть NaN або значення відсутнє, та False в інших випадках.

[https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isna.html](http://)

In [ ]:
sns.heatmap(data.isna())
plt.show()

Як видно з графіка, всі значення дорівнюють 0(False), отже пропущених значень та NaN у датасеті немає, тому додаткова обробка на цьому етапі непотрібна.

# 2. Провести візуалізацію даних. Побудувати такі типи графіків: Line Plot, Bar Chart, Histogram, Heatmap. При цьому для візуалізації багатовимірних даних з кількістю ознак більше двох обов’язково необхідно за допомогою subplot збудувати 9 графіків за допомогою комбінування різних ознак.

Графіки **Line Plot** використовуються для первинного відстеження залежностей між всіма можливими парами (вхід, вихід):

In [ ]:
fig = plt.figure(figsize = (20, 6))

ax_1 = fig.add_subplot(2, 2, 1)
ax_2 = fig.add_subplot(2, 2, 2)
ax_3 = fig.add_subplot(2, 2, 3)
ax_4 = fig.add_subplot(2, 2, 4)

sns.lineplot(data = data, x = "R&D Spend", y = "Profit", ax = ax_1)
sns.lineplot(data = data, x = "Administration", y = "Profit", ax = ax_2)
sns.lineplot(data = data, x = "Marketing Spend", y = "Profit", ax = ax_3)
sns.lineplot(data = data, x = "State", y = "Profit", ax = ax_4)

plt.show()

Аналізуючи перший графік можна помітити майже лінійну залежність між витратами на науково-дослідну роботу та прибутком, менше помітна, але все-таки залежність прослідковується між прибутком та витратами на маркетинг.

На графіках **Bar Chart** зручно аналізувати числові характеристики залежностей:

In [ ]:
fig = plt.figure(figsize = (20, 6))

ax_1 = fig.add_subplot(2, 2, 1)
ax_2 = fig.add_subplot(2, 2, 2)
ax_3 = fig.add_subplot(2, 2, 3)
ax_4 = fig.add_subplot(2, 2, 4)

sns.barplot(data = data, y = "Profit", x = "R&D Spend", ax = ax_1)
sns.barplot(data = data, y = "Profit", x = "Administration", ax = ax_2)
sns.barplot(data = data, y = "Profit", x = "Marketing Spend", ax = ax_3)
sns.barplot(data = data, y = "Profit", x = "State", ax = ax_4)

plt.show()

Використовуючи графік **pairplot** можна візуалізувати відношення між усіма можливими парами змінних:

In [ ]:
sns.pairplot(data, hue = 'State')
plt.show()

За допомогою метода **corr()** та графіка **heatmap** можна переконатися у правдивості знайдених раніше залежностей та оцінити кореляцію між всіма парами параметрів :

In [ ]:
sns.heatmap(data = data.corr(), linewidths = .5, annot = True, cmap = "YlGnBu")
plt.show()

Числові значення цієї матриці - коефіцієнти кореляції Пірсона.

Коефіцієнт кореляції Пірсона - це коефіцієнт лінійної залежності між двома змінними, що набуває значення від -1 до 1. При цьому при від’ємному значенні коефіцієнту Пірсона ми говоримо про зворотну залежність, а при низьких по модулю значеннях ми говоримо про слабку або відсутню залежність.

In [ ]:
img(filename = '../input/imaages/2.png')

Як і очікувалось, R&D Spend та Marketing Spend мають високу кореляцію з Profit.

# 3.Вирахувати математичне сподівання  та дисперсію σ для набору даних. Стандартизувати дані.

Стандартизація даних[1] відбувається за наступною формулою:

In [ ]:
img(filename = '../input/imaages/3.png')

In [ ]:
titles = ['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit']

for i in titles:
    print(f'\nматематичне сподівання {i} =', round(data[f'{i}'].mean(), 3))
    print(f'дисперсія σ {i} =', round(data[f'{i}'].std(), 3))

print('\n')

Програмно дані стандартизуються за допомогою метода **StandardScaler()**:

[https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html](http://)

In [ ]:
data = pd.DataFrame(StandardScaler().fit_transform(data)).rename(columns = 
    {0: titles[0], 1: titles[1], 2: titles[2], 3: titles[3], 4: titles[4]})

Датасет після стандартизації:

In [ ]:
data.head(6)

Математичне сподівання та дисперсія σ даних для перевірки успішності процесу стандартизації:

In [ ]:
titles = ['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit']

for i in titles:
    print(f'\nматематичне сподівання {i} =', abs(round(data[f'{i}'].mean(), 3)))
    print(f'дисперсія σ {i} =', round(data[f'{i}'].std(), 3))

print('\n')

# 4.	За допомогою Random Forest моделі визначити інформативність признаків.

Використовуючи модель Random Forest, можна виміряти інформативність ознаки[2]. Для цього необхідно навчити модель на вибірці і під час побудови моделі для кожного елемента навчальної вибірки порахувати out-of-bag-помилку за наступною формулою :

In [ ]:
img(filename = '../input/imaages/4.png')

Потім для кожного об'єкта така помилка усереднюється по всьому випадковому лісу. Щоб оцінити інформативність ознаки, її значення перемішуються для всіх об'єктів навчальної вибірки і out-of-bag-помилка рахується знову. Важливість ознаки оцінюється шляхом усереднення по всіх деревах різниці показників out-of-bag-помилок до і після перемішування значень.

Програмно інформативність ознак визначається за допомогою метода **feature_importances** класа **RandomForestRegressor**:

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html](http://)

In [ ]:
X = data.drop('Profit', axis = 1)
Y = data['Profit']

forest = RandomForestRegressor().fit(X, Y)
importances = forest.feature_importances_

titles = ['R&D Spend', 'Administration', 'Marketing Spend', 'State']
for i in range(len(titles)):
    print(round(importances[i], 5), f'- інформативність {titles[i]}')

# 5. Зменшити розмірність даних за допомогою методу Principal Components Analysis (PCA) до двох вимірів та провести візуалізацію, що вимагається в пункті 2.

**Аналіз головних компонент (pгincipal component analysis, РСА)**([1], [3]) - це метод лінійного перетворення, що належить до типу навчання без учителя, який широко використовується в самих різних областях, найчастіше для зниження розмірності. **РСА** допомагає ідентифікувати повторювані образи в даних, ґрунтуючись на кореляції між ознаками. Якщо коротко, то **РСА** знаходить напрямки з максимальною дисперсією в багатовимірних даних і проектує їх на новий підпростір з меншим числом розмірностей, ніж вихідне. Ортогональні осі нового підпростору можна інтерпретувати як напрямки максимальної дисперсії в умовах обмеження, що осі нових признаків ортогональні.

Спочатку для стандартизованого d-вимірного набору даних вираховуєтсья коваріаційна матриця. Після цього відбувається розклад матриці на власні вектори та власні числа. Наступним кроком є вибір k власних векторів які відповідають k найбільшим власним значенням, де k - кількість вимірів у новому просторі даних. Оскільки власні числа відповідають довжині власних векторів, вибравши найбільші в якості осей для нового простору, ми залишаємо найбільшу частину дисперсії, що в свою чергу зменшує кількість втраченої інформації. Останнім кроком є створення проекційної матриці W, із k найбільших векторів та перепроектування d-вимірного набору даних у k-вимірний.

[https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html](http://)

In [ ]:
X_PCA_pl = PCA(n_components = 2).fit_transform(X)
PCA_data_pl = pd.DataFrame(data = X_PCA_pl, columns = ['PC1', 'PC2'])
PCA_data_pl['Profit'] = data['Profit']

Датасет після зменешення розмірності даних:

In [ ]:
PCA_data_pl.head(6)

Візуалізація даних:

In [ ]:
fig = plt.figure(figsize = (20, 6))

ax_1 = fig.add_subplot(1, 2, 1)
ax_2 = fig.add_subplot(1, 2, 2)

sns.lineplot(data = PCA_data_pl, x = "PC1", y = "Profit", ax = ax_1)
sns.lineplot(data = PCA_data_pl, x = "PC2", y = "Profit", ax = ax_2)


plt.show()

In [ ]:
fig = plt.figure(figsize = (20, 6))

ax_1 = fig.add_subplot(1, 2, 1)
ax_2 = fig.add_subplot(1, 2, 2)

sns.barplot(data = PCA_data_pl, y = "Profit", x = "PC1", ax = ax_1)
sns.barplot(data = PCA_data_pl, y = "Profit", x = "PC2", ax = ax_2)


plt.show()

In [ ]:
sns.pairplot(PCA_data_pl)
plt.show()

In [ ]:
sns.heatmap(data = PCA_data_pl.corr(), linewidths = .5, annot = True)
plt.show()

# Вирахувати відсотки дисперсії для власних векторів та визначити скільки потрібно залишити власних векторів, щоб залишилося більше 90% дисперсії.

Для визначення відсотків дисперсії власних векторів, метод **PCA** використовується для датасету без вказання кількості компонент:

In [ ]:
pca = PCA(n_components = None)
_ = pca.fit(X)

Власні числа коваріаційної матриці можна дізнатися за дпомогою методу **explained__variance__**:

In [ ]:
print(pca.explained_variance_)

Відсотки дисперсії для власних векторів визначаються методом **explained__variance__ratio__**:

In [ ]:
print(pca.explained_variance_ratio_)

Тому, для збереження > 90% дисперсії необхідно залишити 3 власних вектора.

In [ ]:
X_PCA = PCA(n_components = 3).fit_transform(X)
PCA_data = pd.DataFrame(data = X_PCA, columns = ['PC1', 'PC2', 'PC3'])
PCA_data['Profit'] = data['Profit']

датасет після зменешення розмірності даних до 3:

In [ ]:
PCA_data.head(6)

# 6.	Оцінити залежність між набором даних та набором предикторів за допомогою коефіцієнту Пірсона.

In [ ]:
sns.heatmap(data = PCA_data.corr(), linewidths = .5, annot = True)
plt.show()

Значення коефіцієнтів Пірсона зменшились в порівнянні з початковими даними, шо не дивно, адже початкова розмірність даних невелика, тому для подальшої роботи залишається стандартизований датасет без використання **РСА**.

# 7.	Використати правило 68-95-99.7 для набору даних, визначити чи розподіл даних є подібним до нормального розподілу. Для цього вирахувати:
# a.	Pr( µ-1σ ≤ X ≤ µ+1σ)
# b.	Pr( µ-2σ ≤ X ≤ µ+2σ)
# c.	Pr( µ-3σ ≤ X ≤ µ+3σ)

При вимірюванні випадкова величина X з математичним сподіванням µ та дисперсією σ приймає випадкові значення. Для випадкової величини з нормальним розподілом діє правило[4]:

In [ ]:
img(filename = '../input/imaages/5.png')

Отже за допомогою цього правила можна знаходити викиди, як всі елементи, що не належать інтервалу  µ-3σ; µ+3σ.

In [ ]:
titles = ['PC1', 'PC2', 'PC3', 'Profit']

for i in titles:
    m = PCA_data[f'{i}'].mean()
    s = PCA_data[f'{i}'].std()
    val = [0, 0, 0]
    for j in PCA_data[f'{i}'].values:
        if j > m - s and j < m + s:
            val[0] += 1
        if j > m - 2 * s and j < m + 2 * s:
            val[1] += 1
        if j > m - 3 * s and j < m + 3 * s:    
            val[2] += 1
    print(f'\nPr⁡(µ - 1σ ≤ {i} ≤ µ + 1σ) =', val[0] / 50)
    print(f'Pr⁡(µ - 2σ ≤ {i} ≤ µ + 2σ) =', val[1] / 50)
    print(f'Pr⁡(µ - 3σ ≤ {i} ≤ µ + 3σ) =', val[2] / 50)

print('\n')

# 8.Видалити з набору даних всі дані, що не належать до інтервалу  µ-3σ; µ+3σ

Для даного датасета, в кожному наборі всі дані знаходяться в межах інтервалу  µ-3σ; µ+3σ, тому додаткова обробка даних на цьому етапі непотрібна. 

# 9.	Провести тест Колмогорова-Смірнова.

Критерій узгодженості Колмогорова-Смірнова призначений для перевірки гіпотези про приналежність вибірки деякому закону розподілу, тобто перевірки того, що емпіричний розподіл відповідає передбачуваної моделі. Тест Колмогорова-Смiрнова[5] показує наскiльки значущими є вiдмiнностi мiж двома розподiлами випадкових величин. В нашому випадку, розподiл ознаки порівнюється з нормальним розподiлом. Для цього вiдбувається пiдрахунок статистики Колмогорова-Смiрнова:

In [ ]:
img(filename = '../input/imaages/6.png')

Для достатньо великого датасету гiпотеза відкидається на рівні значущості α при умові:

In [ ]:
img(filename = '../input/imaages/7.png')

Тест Колмогорова-Смірнова проводиться за допомогою функції **kstest**:

[https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html](http://)

In [ ]:
def check_test_result(statistic, distribution = 'norm', alpha = 0.05):
    critical_value = (1 / np.sqrt(50)) * np.sqrt(-np.log(alpha / 2))
    if statistic < critical_value:
        return f'гіпотеза про належність виборки до розподілу {distribution} приймається на рівні значущості α = {alpha}'
    else:
        return f'гіпотеза про належність виборки до розподілу {distribution} відкидається на рівні значущості α = {alpha}'

In [ ]:
res = kstest(PCA_data['PC1'], 'norm').statistic
print('D =', res)
print(check_test_result(res))

In [ ]:
res = kstest(PCA_data['PC2'], 'norm').statistic
print('D =', res)
print(check_test_result(res))

In [ ]:
res = kstest(PCA_data['PC3'], 'norm').statistic
print('D =', res)
print(check_test_result(res))

In [ ]:
res = kstest(PCA_data['Profit'], 'norm').statistic
print('D =', res)
print(check_test_result(res))

# Реалізувати та навчити різні моделі з різною архітектурою для розв’язання поставленої задачі регресії. 

In [ ]:
from sklearn.svm import SVR
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5, random_state = 13)

Для розв’язання задачі використовуються всі моделі, які були розглянуті протягом семестру, при чому для наглядності та об'єктивності, їх параметри, за винятком можливих непередбачуваних ситуацій, встановлені за замовчуванням. 

Для оцінки моделей використовується коефіцієнт детермінації - статистичний показник, що використовується в статистичних моделях як міра залежності варіації залежної змінної від варіації незалежних змінних. Вказує наскільки отримані спостереження підтверджують модель. Коефіцієнт детермінації визначається наступним чином:

In [ ]:
img(filename = '../input/imaages/8.png')

In [ ]:
model_titles = ['Lasso', 'Ridge', 'Elnet', 'KNN', 'MLP', 'SVR', 'RanFor']
train_error = []
test_error = []

Метод регресії «Lasso» полягає у введенні додаткової складової регулювання в функціонал оптимізації моделі, що часто дозволяє отримувати більш стійке рішення. Умова мінімізації квадратів помилки при оцінці параметрів β виражається наступною формулою:

[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html](http://)

In [ ]:
img(filename = '../input/imaages/9.png')

In [ ]:
lasso_regression = Lasso()

lasso_regression.fit(X_train, Y_train)
lasso_result = lasso_regression.predict(X_test)

train_err = 1 - lasso_regression.score(X_train, Y_train)
test_err = 1 - lasso_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, lasso_regression.predict(X_test), squared = False))

Метод регресії «Ridge» відрізняється від моделі Lasso тільки експонентою при параметрі регуляризації. Умова мінімізації квадратів помилки при оцінці параметрів β виражається наступною формулою:

[https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html](http://)

In [ ]:
img(filename = '../input/imaages/10.png')

In [ ]:
ridge_regression = Ridge()

ridge_regression.fit(X_train, Y_train)
ridge_result = ridge_regression.predict(X_test)

train_err = 1 - ridge_regression.score(X_train, Y_train)
test_err = 1 - ridge_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, ridge_regression.predict(X_test), squared = False))

Метод регресії «Elastic Net» являється узагальненням регресії з регуляризацією, вона об'єднує Ridge регресію (при λ2 = 0) і регресію Lasso (при λ1 = 0). Умова мінімізації квадратів помилки при оцінці параметрів β виражається наступною формулою:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html

In [ ]:
img(filename = '../input/imaages/11.png')

In [ ]:
elnet_regression = ElasticNet()

elnet_regression.fit(X_train, Y_train)
elnet_result = elnet_regression.predict(X_test)

train_err = 1 - elnet_regression.score(X_train, Y_train)
test_err = 1 - elnet_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, elnet_regression.predict(X_test), squared = False))

Ключова ідея методу, що лежить в основі «KNN», полягає в формулюванні моделі в термінах евклідових відстаней в вихідному багатовимірному просторі ознак. Завдання полягає в тому, щоб для кожної тестованої точки знайти такий δ-окіл, щоб в ній помістилося k точок з відомими значеннями y. Тоді прогноз можна отримати, усереднюючи значення всіх навчальних спостережень з δ.

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html

In [ ]:
knn_regression = KNeighborsRegressor()

knn_regression.fit(X_train, Y_train)
knn_result = knn_regression.predict(X_test)

train_err = 1 - knn_regression.score(X_train, Y_train)
test_err = 1 - knn_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, knn_regression.predict(X_test), squared = False))

Багатошаровий персептрон (MLP) - це алгоритм навчання з учителем, складається щонайменше з трьох шарів: вхідного шару, прихованого шару та вихідного шару. За винятком вхідних вузлів, кожен вузол є нейроном, який використовує нелінійну функцію активації. MLP використовує контрольовану техніку навчання, яка називається зворотним поширенням помилки для навчання. Його багатошаровість і нелінійна активація дозволяють розрізняти дані, які не є лінійно роздільними.

[https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html](http://)

In [ ]:
img(filename = '../input/imaages/12.png')

In [ ]:
mlp_regression = MLPRegressor()

mlp_regression.fit(X_train, Y_train)
mlp_result = mlp_regression.predict(X_test)

train_err = 1 - mlp_regression.score(X_train, Y_train)
test_err = 1 - mlp_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, mlp_regression.predict(X_test), squared = False))

Метод «SVM» - алгоритм навчання з учителем. Основна ідея методу - переведення вихідних векторів у простір більш високої розмірності і пошук розділяючої гіперплощини з максимальним зазором у цьому просторі. Дві паралельні гіперплощини будуються по обі сторони від гіперплощини. Гіперплощина, що розділяє, максимізує відстань до двох паралельних гіперплощин. Чим більше розбіжність або відстань між цими паралельними площинами - тим менше буде середня помилка.

[https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html](http://)

In [ ]:
img(filename = '../input/imaages/13.png')

In [ ]:
svm_regression = SVR()

svm_regression.fit(X_train, Y_train)
svm_result = svm_regression.predict(X_test)

train_err = 1 - svm_regression.score(X_train, Y_train)
test_err = 1 - svm_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, svm_regression.predict(X_test), squared = False))

Random forest - алгоритм машинного навчання, основна ідея полягає в використанні великого ансамблю дерев рішень, кожне з яких саме по собі дає не дуже високу якість передбачення, але за рахунок їх великої кількості результат виходить хорошим. Дерево рішень будує регресійні моделі у вигляді деревної структури. Він розбиває набір даних на все менші та менші підмножини, в той час, як пов'язане дерево рішень поступово розробляється. Кінцевим результатом є дерево з вузлами прийняття рішень та вузлами листів. Вузол прийняття рішень має дві або більше гілок, кожна з яких представляє значення для перевіреного атрибута. Вузол листа представляє рішення щодо числової цілі. Вузол найвищого рішення у дереві, який відповідає найкращому предиктору, називається кореневим вузлом.

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html](http://)

In [ ]:
img(filename = '../input/imaages/14.png')

In [ ]:
forest_regression = RandomForestRegressor()

forest_regression.fit(X_train, Y_train)
forest_result = forest_regression.predict(X_test)

train_err = 1 - forest_regression.score(X_train, Y_train)
test_err = 1 - forest_regression.score(X_test, Y_test)

train_error.append(train_err)
test_error.append(test_err)

print('train error =', train_err, '\ntest error =', test_err)
print('RMSE error =', mean_squared_error(Y_test, forest_regression.predict(X_test), squared = False))

In [ ]:
model_data = pd.DataFrame(data = {'name': model_titles, 'train error': train_error, 'test error': test_error})

Моделі та їх результати представлені у наступному датасеті:

In [ ]:
model_data.head(7)

# На основі аналізу їх результатів вибрати найкращі моделі та ансамблювати їх для отримання більш стійкого результату.

Для подальшого використання залишаються тільки моделі з помилкою на основі коефіцієнта детермінації на тестових даних < 0.1 :

In [ ]:
model_data.drop(model_data[model_data['test error'] >= 0.1].index, inplace = True)

In [ ]:
model_data.head()

Отже, найкращі моделі для розв'язання даної задачі - **Ridge**, **MLPRegressor** та **RandomForestRegressor**.

Ансамблювання моделей використовуючи **VotingRegressor()**:

[https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html](http://)

In [ ]:
ensemble = VotingRegressor([('Ridge', ridge_regression), ('MLP', mlp_regression), ('RanFor', forest_regression)])

ensemble.fit(X_train, Y_train)
ensemble_result = ensemble.predict(X_test)

train_err = 1 - ensemble.score(X_train, Y_train)
test_err = 1 - ensemble.score(X_test, Y_test)

print('train error =', train_err, '\ntest error =', test_err)

In [ ]:
plt.figure()
plt.plot(ridge_result, 'gd', label = 'Ridge')
plt.plot(mlp_result, 'b^', label = 'MLP')
plt.plot(forest_result, 'ys', label = 'RanFor')
plt.plot(ensemble_result, 'r*', ms = 10, label = 'VotingRegressor')

plt.tick_params(axis = 'x', which = 'both', bottom = False, top = False, labelbottom = False)
plt.ylabel('прогноз')
plt.xlabel('тестова вибірка')
plt.legend(loc = "best")

plt.show()

# Аналіз результатів

# 1.	Вплив відновлення пропущених значень порівняно з видаленням їх з вибірки.

Вибраний датасет не містив пропущені значення або NaN, але під час виконання лабораторних робіт практично було встановлено що краще відновити пропущені значення, аніж видаляти їх.

# 2.	Аналіз побудованих графіків та гістограм для різних ознак.

З Linear- та Bar-plots видно, що R&D Spend та Marketing є більш ваговими ознаками ніж Administration та State, а на графіку pairplot можна помітити що Profit залежить від R&D Spend майже лінійно, що підтверджується кореляційною матрицею.

# 3.	Аналіз отриманих даних після застосування алгоритму зменшення розмірності.

Після зменшення розмірності загальні тенденції не змінилися, PC1 візуально складає з Profit таку ж майже лінійну залежність, як і R&D Spend до застосування алгоритма РСА.

# 4.	Аналіз впливу зменшення розмірності даних на точність моделі.

Зменшення розмірності негативно вплинуло на точність всіх моделей, похибка на навчальних та тестових вибірках зросла майже на 20%, тому моделі навчалися на повних, але стандартизованих даних.

# 5.	Аналіз залежності між предикторами та досліджуваною величиною.

Кореляційна матриця повністю підтверджує результати обробки даних, отриманих на етапі візуалізації: коефіцієнт Пірсона між Profit та R&D Spend і Marketing становить відповідно 0.97 та 0.75, що свідчить про високу кореляцію, коефіцієнт Пірсона між Profit та Administration і State становить відповідно 0.2 та 0.1, що свідчить про низьку кореляцію.

# 6.	Аналіз розподілу вхідних даних, аналіз викидів, після переведення його до нормального, за правилом 68-95-99.7

Розподіл початкових вхідних даних не схожий на типові статистичні розподіли, після стандартизації даних, розподілом став стандартно-нормальний, після використання правила 3-сігма не було визначено та вилучено жодного викида, тому що значення всіх признаків знаходяться в межах µ - 3σ ; µ + 3σ.

# 7.	Аналіз результатів тесту Колмогорова-Смірнова.

Тест Колмогорова-Смірнова для кожного набору даних, як і очікувалось, підтвердив гіпотезу про приналежність даних до стандартно-нормального розподілу на рівні значущості 0.05.

# 8.	Як підрахунок різних точностей описує поведінку навчених моделей.

Для оцінювання точності моделей використовувались коефіцієнт детермінації та cередньоквадратична похибка, в процентному співвідношенні для кожної моделі відношення оцінок знаходяться на приблизно однакових рівнях,що дозволяє зробити висновок при прийнятність обох методів оцінки.

# 9.	Вибір оптимальних параметрів моделі, їх обґрунтування.

Для об'єктивності тестування моделей всі без винятку параметри були встановлені за замовчуванням, але для даної задачі ці параметри майже завжди і є оптимальними.

# 10.	Оцінка помилок на начальній та тестовій вибірках.

Похибка на навчальній та тестовій виборках виводиться під кожною нодою, що містить модель.

# 11.	Вибір найкращих моделей для ансамблювання на основі їх точностей та вибір всіх моделей для ансамблювання.

Після аналізу результатів в якості найкращих моделей були обрані Ridge, MLPRegressor та RandomForestRegressor. Їх ансамблювання дозволило отримати більш стійкий та зважений результат, який виявився кращим ніж точність всіх моделей поодинці. Ансамблювання всіх моделей, на нашу думку, не має сенсу, в умовах тих результатів, які ми отримали.

# 12.	Порівняння точності ансамблю та найкращої моделі.

Точність найкращої моделі, Ridge:

In [ ]:
print(1 - model_data.iloc[0]['test error'])

Точність ансамблю:

In [ ]:
print(ensemble.score(X_test, Y_test))

Отже, різниця в точності складає:

In [ ]:
diff = ensemble.score(X_test, Y_test) + model_data.iloc[0]['test error'] - 1
print(diff)

Різниця в точності у відсотках:

In [ ]:
print(round(100 * diff / model_data.iloc[0]['test error']))

# Висновки

Отже, під час виконання курсової роботи, ми навчилися знаходити й позбуватися від пропущених значень в даних, візуалізувати та знаходити залежності даних. Також розібрались в одному з можливих способів визначання інформативності признаків, за допомогою моделі Random Forest, та у методі пониження розмірності даних - PCA. Навчились оцінювати залежності між набором даних та набором предикторів використовуючи коефіцієнт Пірсона, а також проводити тест Колмогорова-Смірнова. Пересвідчились в коректності й використали правило 68-95-99.7.

Ми провели значний аналіз всіх результатів, тим самим навчилися виявляти важливі фактори, які впливають на точність результату, та навпаки
фактори які є негативними для нас. Ці, і не тільки ці навички, які ми здобули виконуючи курсовую роботу, допомогли нам глибше розібратися в актуальних методах машинного навчання, а також в ефективному аналізу даних, що є цінним досвідом для майбутніх проєктів.

# Література

1. Себастьян Рашка - «Python и машинное обучение»
1. Рысьмятова Анастасия - «Методы отбора признаков»
1. Ian T. Jolliffe and Jorge Cadima - «Principal component analysis: a review and recent developments»
1. Michael Galarnyk - «Explaining the 68-95-99.7 rule for a Normal Distribution»
1. Queen Mary's School of Mathematical Sciences - «Kolmogorov-Smirnov Tests»